<!-- 
---

### Promemoria: questo 📘 `.NET Interactive` deve essere eseguito da VS Code con [questi prerequisiti](../PREREQS.md).

--- -->

<!-- 
#### How to use this notebook: 

* Just read the text and scroll along until you run into code blocks.
* Code blocks have computer code inside them — hover over the block and you can run the code.
* Run the code by hitting the ▶️ "play" button to the left. If the code runs you'll see a ✔️. If not, you'll get a ❌.
* The output and status of the code block will appear just below itself — you need to scroll down further to see it.
* Sometimes a code block will ask you for input in a hard-to-notice dialog box 👆 at the top of your notebook window.  -->

<!-- --- -->

# Recipe III: 🛢️ Skills

## Fun Fact 1. 🛢️ Le Semantic skills sono 📁 cartelle di ⚡ funzioni

Il modo in cui si accede a una funzione semantica è attraverso il nome dell'abilità. Come `A-Skill.A-Function` o `A-Skill.Another-Function.`.

```
🗂️ skills
│
└─── 📁 A-Skill
│    └─── ⚡ A-Function 
│    └─── ⚡ Another-Function   
│
└─── 📁 B-Skill
│    └─── ⚡ Function-B1   
│    └─── ⚡ Function-B2   
│
└─── 📁 C-Skill
     └─── ⚡ Function-C1   

```

## Fun Fact 2. 🛢️ Le Funzioni Semantiche sono 📄 file di testo

Questo `skprompt.txt` di cui abbiamo appena parlato, è la nostra Funzione Semantica!

```
📁 A-Skill
│
└─── ⚡ A-Function
|    |
│    └─── 📄 skprompt.txt
│   
└─── ⚡ Another-Function
     |
     └─── 📄 skprompt.txt
```
Il formato del file `config.json` è possibile visonarlo [qui](https://learn.microsoft.com/en-us/semantic-kernel/howto/configuringfunctions) ma se non è incluso, verrà utilizzato un comportamento predefinito.

## Step 1: Iniziare con l'istanziazione di un kernel 🔥

Le informazioni sulla chiave API sono già state impostate, quindi basta premere ▶️ (play) e il gioco è fatto.

In [1]:
#r "nuget: Microsoft.SemanticKernel, 0.9.61.1-preview"

#!import ../config/Settings.cs

using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.KernelExtensions;
using System.IO;
using Microsoft.SemanticKernel.Configuration;
using Microsoft.SemanticKernel.SemanticFunctions;

IKernel kernel = Microsoft.SemanticKernel.Kernel.Builder.Build();

// Grab the locally stored credentials from the settings.json file. 
// Name the "backend" as "davinci" — assuming that you're using one of the davinci completion models. 

var (useAzureOpenAI, model, azureEndpoint, apiKey, orgId) = Settings.LoadFromFile();

if (useAzureOpenAI)
    kernel.Config.AddAzureOpenAITextCompletion("davinci", model, azureEndpoint, apiKey);
else
    kernel.Config.AddOpenAITextCompletion("davinci", model, apiKey, orgId);

Installed Packages Microsoft.SemanticKernel, 0.9.61.1-preview

😱 **Ricevete un messaggio di errore?** Nel [primo notebook](../s1e1-ez-starter-notebook/notebook.ipynb) abbiamo visto nel dettaglio questo passaggio.

## Step 2: 🛢️ Accedere a una funzione semantica ⚡ dalla sua 📁 cartella delle skill

Utilizzeremo la funzione semantica `Joke` che è accessibile all'interno della sottodirectory `FunSkill` di `skills.`\
Questa funzione può essere indicata come `FunSkill.Joke` per indicare come la funzione `Joke` si trovi all'interno di `FunSkill.`
```
🗂️ skills
│
└─── 📁 FunSkill
|    |
│    └─── ⚡ Joke
│    └─── ⚡ Limerick
│    └─── ⚡ Excuses
│
└─── 📁 ChatSkill
```

### Step 2.1:🛢️ Raccontami una barzelletta usando la funzione semantica `FunSkill.Joke`.

E con questo, chiediamo all'AI LLM di raccontarci una barzelletta 😂.

In [3]:
// Load the Skills Directory
var skillsDirectory = Path.Combine(System.IO.Directory.GetCurrentDirectory(), "skills");

// Load the FunSkill from the Skills Directory
var mySkill = kernel.ImportSemanticSkillFromDirectory(skillsDirectory, "FunSkill");

// The default input variable 
var myInput = "time travel to dinosaur age";

// Run the Function called Joke with the default parameter of $input
var result = await kernel.RunAsync(myInput, mySkill["Joke"]);

// Return the result to the Notebook
Console.WriteLine(result);


Q: What did the time traveler say when they arrived in the dinosaur age?
A: "Dinosaurs, I thought you'd be bigger!"


### Step 2.2: 🛢️ Fornire più input a una funzione semantica

Ci sono in realtà due parametri utilizzati da `FunSkill.Joke`.\
Uno è `$input` e l'altro è `$audience_type`.\
Possiamo fornire questo contesto più ampio alla funzione semantica, assegnando le `ContextVariables`, e poi eseguire di nuovo la funzione semantica con il contesto più ampio rispetto al singolo `$input`:

In [4]:
using Microsoft.SemanticKernel.Orchestration;

// Reload the FunSkill from the Skills Directory in case you are changing it for fun
var mySkill = kernel.ImportSemanticSkillFromDirectory(skillsDirectory, "FunSkill");

// THIS IS NEW!
var myContext = new ContextVariables(); 

// The variables are manually set when you use a ContextVariables object
myContext.Set("input", "going to the department store"); 
myContext.Set("audience_type", "snobby people"); 

var myResult = await kernel.RunAsync(myContext,mySkill["Joke"]);

Console.WriteLine(myResult);


A snobby woman went to the department store and asked the clerk, "Do you have any clothes that don't make me look fat?" The clerk replied, "No ma'am, we don't carry any clothes that can do that."



### Step 2.3: 🛢️ Ogni funzione semantica non ha bisogno di usare `$input`.

L'uso della variabile di contesto predefinita `$input` è facoltativo.\
Ad esempio, `FunSkill.Limerick` ha due variabili di contesto `$name` e `$whoisname` da usare come segue:

In [5]:
using Microsoft.SemanticKernel.Orchestration;

// Load the Limerick function from the FunSkill

var mySkill = kernel.ImportSemanticSkillFromDirectory(skillsDirectory, "FunSkill");

var myContext = new ContextVariables(); 
myContext.Set("name", "Marie Curie"); 
myContext.Set("who_is_name", "the great scientist"); 

var myResult = await kernel.RunAsync(myContext,mySkill["Limerick"]);

Console.WriteLine(myResult);


There once was a scientist named Marie,
Who studied and worked with great glee.
She found radium and more,
And her fame did soar,
Making science history!


# ⏭️ I prossimi passi

Esempi più avanzati nei notebook disponibili nel nostro repo GitHub all'indirizzo [https://aka.ms/sk/repo](https://aka.ms/sk/repo).

[Vediamo le 🍝 memories!](../e4-memories/notebook.ipynb)